In [29]:
import tensorflow
import gym
import random
import atari_py
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy



from gym import Env
from gym.spaces import Discrete, Box
import random

In [7]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 38 + random.randint(-3,3)
        self.shower_lenght =60


    def step(self, action):
        self.state += (action -1)*2
        self.shower_lenght -=1


        if self.state >= 37 and self.state <= 39:
            reward =1
        else:
            reward = -1

        if self.shower_lenght <= 0:
            done = True
        else:
            done = False

        self.state += random.randint(-1,1)
        info = {}

        return self.state, reward, done, info

    def render(self):
        pass
    def reset(self):
        self.state = 38 + random.randint(-3,3)
        self.shower_lenght =60
        return self.state

    

In [8]:
env = ShowerEnv()

C:\Users\bartw\.conda\envs\games_rl\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [17]:
env.action_space.sample()

0

In [18]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} score:{}'.format(episode, score))
env.close()

Episode:1 score:-54
Episode:2 score:-58
Episode:3 score:-16
Episode:4 score:-58
Episode:5 score:-22
Episode:6 score:-46
Episode:7 score:-50
Episode:8 score:-58
Episode:9 score:-8
Episode:10 score:-50


In [19]:
states = env.observation_space.shape
actions = env.action_space.n

In [33]:
def build_model(states, actions):
    model = Sequential()
    
    model.add(Dense(24, activation='relu', input_shape = states))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model


In [34]:
model = build_model(states, actions)



In [37]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),
                                  attr="eps", value_max = 1.,
                                  value_min = .1,
                                  value_test = 2,
                                  nb_steps = 10000
                                 )
    
    memory = SequentialMemory(limit =100, window_length =1)
    
    dqn = DQNAgent(model = model,
                   memory=memory,
                   policy=policy, 
                   enable_dueling_network = True,
                   dueling_type = 'avg',
                   nb_actions = actions,
                   nb_steps_warmup =1000
                  )
    return dqn

    
    

In [38]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))
dqn.fit(env, nb_steps = 10000, visualize=False, verbose =1)

Training for 10000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
10000/10000 [==============================] - 42s 4ms/step - reward: -0.7130
done, took 41.565 seconds
